In [ ]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
###### helper functions. Use them when needed #######
def get_title_from_index(index):
	return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
	return df[df.title == title]["index"].values[0]
##################################################

In [1]:
df=pd.read_csv(r"\Desktop\movie_dataset.csv")

In [ ]:
import os
import time
import playsound
import speech_recognition as sr
from gtts import gTTS
import pyttsx3

In [ ]:
import pytz

In [ ]:
import subprocess

In [ ]:
def speak(text,mm):
  tts=gTTS(text=text,lang='en',slow=False)
  filename='voice'+str(mm)+'.mp3'
  tts.save(filename)
  playsound.playsound(filename)

In [ ]:
def get_audio():
    r=sr.Recognizer()
    with sr.Microphone() as source:
        audio=r.listen(source)
        said=''
        
        try:
            said=r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception " + str(e))
    return said.lower()

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
DAYS=['monday','tuesday','wednesday','thursday','friday','saturday','sunday']
MONTHS=['january','february','march','april','may','june','july','august','september','october','november','december']
days_extension=['rd','th','st','nd']

In [ ]:
if 'hello' in text:
    speak('Hello, so nice of you to talk with me')

In [ ]:
if 'what is your name' in text:
    speak('My name is Alexa')

In [ ]:
def authenticate_google():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    
    return service

def get_events(day,service):
    date=datetime.datetime.combine(day,datetime.datetime.min.time())
    end_date=datetime.datetime.combine(day,datetime.datetime.max.time())
    utc=pytz.UTC
    date=date.astimezone(utc)
    end_date=end_date.astimezone(utc)
    
    events_result = service.events().list(calendarId='primary', timeMin=date.isoformat(),timeMax=end_date.isoformat(),
                                         singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        speak('No upcoming events found.')
    
    else:
        speak(f'You have {len(events)} events on this day.',98090)
        count=0
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            start_time=str(start.split('T')[1].split("-")[0])
            if int(start_time.split(":")[0])<12:
                start_time=start_time+'AM'
            elif int(start_time.split(":")[0])==12:
                start_time=start_time.split(":")[0]+':'+start_time.split(":")[1]+' PM'
            else:
                start_time=str(int(start_time.split(":")[0])-12)+':'+start_time.split(":")[1]
                start_time=start_time+' PM'
            count=count+1
            speak(event['summary']+' at '+start_time,count)
            print(event['summary']+' at '+start_time)

In [ ]:
def get_date(text):
    text=text.lower()
    today=datetime.date.today()
    
    if text.count('today')>0:
        return today
    
    day=-1
    day_of_week=-1
    month=-1
    year=today.year
    
    for word in text.split():
        if word in MONTHS:
            month=MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week=DAYS.index(word)
        elif word.isdigit():
            day=int(word)
        else:
            for ext in days_extension:
                found=word.find(ext)
                if found>0:
                    try:
                        day=int(word[:found])
                    except:
                        pass
        
    if month<today.month and month!=-1:
        year=year+1
    
    if day<today.day and month ==-1 and day!=-1:
        month=month+1
    
    if month==-1 and day==-1 and day_of_week !=-1:
        current_day_of_week=today.weekday()
        dif=day_of_week - current_day_of_week
        
        if dif<0:
            dif+=7
            if text.count('next')>=1:
                dif=dif+7
        return today+datetime.timedelta(dif)
    if month==-1 or day==-1:
        return None
    return datetime.date(month=month,day=day,year=year)

In [ ]:
def note(text):
    date=datetime.datetime.now()
    file_name= str(date).replace(':','-') + '-note.txt'
    with open(file_name,'w') as f:
        f.write(text)
    
    sublime='C:\Program Files\Sublime Text 3\sublime_text.exe'
    subprocess.Popen(['notepad.exe',file_name])

In [ ]:
# Making a note
note('Akarsh is a good boy')

In [ ]:
speak('Hello Akarsh, My name is Alexa',98765)

In [ ]:
WAKE='hey alexa'

In [ ]:
# Authenticating with Google Calendar
SERVICE=authenticate_google()
print('Start')

Start


In [ ]:
while True:
    print('Listening, Akarsh')
    text=get_audio().lower()
    
    if text.count(WAKE)>0:
        speak('I am ready Akarsh',456789)
        text=get_audio()
        
        
        # Taking Note
        CALENDAR_STRS=['what do i have','do i have plans','am i busy']
        
        # Process for speaking schedule
        for phrase in CALENDAR_STRS:
            if phrase in text.lower():
                date=get_date(text)
                if date:
                  get_events(get_date(text),SERVICE)  
                else:
                    speak('I did not undersyand what you meant, Akarsh',234)
                    
        # Taking Notes
        NOTE_STR=['make a note','write this down','remember this']
        
        # Process for taking the notes
        for phrase in NOTE_STR:
            if phrase in text:
                speak('what would you like me to write down?',23322)
                note_text=get_audio()
                note(note_text)
                speak('I have made a note of that, Akarsh',1234)

Listening, Akarsh
hey Alexa
make a note
Akash is a good boy
Listening, Akarsh
Exception 
Listening, Akarsh


In [ ]:
# Schedule Checking
text=get_audio()

what do I have on Thursday


In [ ]:
# Schedule Speaking
get_events(get_date(text),SERVICE)

Meeting with Elon Musk at 12 PM
Taking Online Lectures at 1:00 PM
Submitting the Engineering Mechanics Assignment at 2:30 PM


In [ ]:
# For speaking the schedule

In [ ]:
# Process for speaking schedule
for phrase in CALENDAR_STRS:
    if phrase in text.lower():
        date=get_date(text)
        if date:
          get_events(get_date(text),SERVICE)  
        else:
            speak('Please try again',234)

Meeting with Elon Musk at 12:00 PM
Taking Online Lectures at 1:00 PM
Submitting the Engineering Mechanics Assignment at 2:30 PM


In [ ]:
# For making the Notes

In [ ]:
# Taking Notes
NOTE_STR=['make a note','write this down','remember this']
for phrase in NOTE_STR:
    if phrase in text:
        speak('what would you like me to write down?',23322)
        note_text=get_audio()
        note(note_text)
        speak('I have made a note of that, Akarsh',1234)